In [2]:
import pandas as pd
import numpy as np
import re
#import dask as d
#import dask.dataframe as dd
#from dask.delayed import delayed

pattern = "^(.*)(?=[0-9a-f]{8}-[0-9a-f]{4}-[1-5][0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12})"

df1 = pd.read_excel("My Report_SAP_R075 2022-09-19 13_03 CEST.xlsx", skiprows = range(0,21), usecols = ['Worker',
                                                                             'Party ID', 
                                                                             'Employee ID',
                                                                             'GUID', 
                                                                             'Cost Center - ID', 
                                                                             'Cost Center - Name', 
                                                                             'Company Reference ID',
                                                                             'Company - Name', 
                                                                             'Management Level', 
                                                                             'Job Level - Primary Position', 
                                                                             'Work Email', 
                                                                             'Status Details', 
                                                                             'Line of Service', 
                                                                             'Hire Date', 
                                                                             'Assignment Reason (Alternative)', 
                                                                             'Host Country (Alternative)', 
                                                                             'Host Country / Territory', 
                                                                             'Country/Territory', 
                                                                             'Company Hierarchy', 
                                                                             'Worker Type', 
                                                                             'Contingent Worker Type', 
                                                                             'Job Category'])
df1 = df1.sort_values('Party ID')
df1['Party ID'] = df1['Party ID'].str.replace(f'{pattern}', '')
df1 = df1[df1['Party ID'].str.len() == 36]
df1 = df1[df1['Contingent Worker Type'] != 'Enterprise BOT']
#df1.to_csv("WD_R075_10.08.csv")


df2 = pd.read_excel("My Report_SAP_R139 2022-09-19 13_02 CEST.xlsx", skiprows = range(0,27), usecols = ['Worker',
                                                                             'Party ID', 
                                                                             'Employee ID', 
                                                                             'GUID', 
                                                                             'Cost Center ID', 
                                                                             'Cost Center Name',
                                                                             'Company - Name', 
                                                                             'Management Level', 
                                                                             'Job Level - Primary Position', 
                                                                             'Email - Primary Work', 
                                                                             'Employee Status', 
                                                                             'Cost Center Level 3', 
                                                                             'Hire Date', 
                                                                             'Termination Date', 
                                                                             'Country/Territory of Work Location', 
                                                                             'Country', 
                                                                             'Worker Type', 
                                                                             'Contingent Worker Type', 
                                                                             'Job Category'])
df2 = df2.sort_values('Party ID')
df2['Party ID'] = df2['Party ID'].str.replace(f'{pattern}', '')
df2 = df2[df2['Party ID'].str.len() == 36]
df2 = df2[df2['Contingent Worker Type'] != 'Enterprise BOT']
#df2.to_csv("WD_R139_10.08.csv")

df3 = pd.read_excel("My Report_SAP_R311 2022-09-19 12_53 CEST.xlsx", skiprows = range(0,20), usecols = ['Worker',
                                                                             'Party ID', 
                                                                             'Employee ID', 
                                                                             'GUID', 
                                                                             'Cost Center ID', 
                                                                             'Cost Center - Name', 
                                                                             'Company Reference ID',
                                                                             'Company - Name', 
                                                                             'Management Level', 
                                                                             'Job Level', 
                                                                             'Email - Primary Work',
                                                                             'Employee Status', 
                                                                             'PwC Global Line Of Service', 
                                                                             'Hire Date', 
                                                                             'Company Hierarchy', 
                                                                             'Cost Center Level 3', 
                                                                             'Worker Type', 
                                                                             'Contingent Worker Type', 
                                                                             'Job Category',
                                                                             'Is Inbound IA? (Exception Process)'])
df3 = df3.sort_values('Party ID')
df3['Party ID'] = df3['Party ID'].str.replace(f'{pattern}', '')
df3 = df3[df3['Party ID'].str.len() == 36]
df3 = df3[df3['Contingent Worker Type'] != 'Enterprise BOT']
#df3.to_csv("WD_R311_10.08.csv")

df4 = pd.read_excel("PMDM_Active_Employee_Report_10.08.xlsx", skiprows = range(0,5), usecols = ['Party ID', 'INAB GUID 1', 'INAB GUID 2', 'Cost Center Code', 'Cost Center Description', 'Legal Entity Party ID','Legal Entity Name', 'Job Level ', 'Grade', 'Cloud Mail', 'Territory', 'Global LoS', 'Contract Type', 'Job Category'])

#df5 = pd.read_excel("RRM_extract_10.08.xlsx")

dfwd1 = pd.merge(df1, df2, how = 'outer', left_on = ['Party ID', 'Employee ID'], right_on = ['Party ID', 'Employee ID'] , indicator = 'Exists_df1_df2')
#dfwd1 = pd.concat([df1,df2]) 

#dfwd2 = pd.merge(dfwd1, df3, how = 'outer', left_on = ['Party ID', 'Employee ID'], right_on = ['Party ID', 'Employee ID'], indicator = 'Exists_dfwd1_df3')
#dfwd2 = pd.concat([dfwd1,df3], ignore_index=True).drop_duplicates() 
#dfwd3 = pd.merge(dfwd2, df4, on = 'Party ID',how = 'outer', indicator='Exists_dfwd2_df4')

dfwdpmdm.to_excel('DataMerge_WD_PMDM.xlsx')

In [3]:
dfwd2 = pd.merge(dfwd1, df3, how = 'outer', left_on = ['Party ID', 'Employee ID'], right_on = ['Party ID', 'Employee ID'],
                 indicator = 'Exists_dfwd1_df3')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
col = dfwd2.pop('Party ID')
dfwd2.insert(0, 'WD_PartyID', col)

col = dfwd2.pop('Employee ID')
dfwd2.insert(1, 'WD_EmployeeID', col)

dfwd2.set_index(['WD_PartyID', 'WD_EmployeeID'])

dfwd2['WD_EmployeeStatus'] = ''
dfwd2.loc[dfwd2['Status Details'] == 'Active', 'WD_EmployeeStatus'] = dfwd2['Status Details']
dfwd2.loc[dfwd2['Status Details'] == 'Future Hire', 'WD_EmployeeStatus'] = dfwd2['Status Details']
dfwd2.loc[dfwd2['Status Details'] == 'Leave of Absence', 'WD_EmployeeStatus'] = dfwd2['Status Details']
dfwd2.loc[dfwd2['Status Details'] == 'Parental Leave', 'WD_EmployeeStatus'] = dfwd2['Status Details']
dfwd2.loc[dfwd2['Status Details'] == 'Secondment', 'WD_EmployeeStatus'] = dfwd2['Status Details']
dfwd2.loc[dfwd2['Employee Status_x'] == 'Active', 'WD_EmployeeStatus'] = dfwd2['Employee Status_x']
dfwd2.loc[dfwd2['Employee Status_x'] == 'Terminated', 'WD_EmployeeStatus'] = dfwd2['Employee Status_x']
dfwd2.loc[dfwd2['Employee Status_y'] == 'Active', 'WD_EmployeeStatus'] = dfwd2['Employee Status_y']
dfwd2.loc[dfwd2['Employee Status_y'] == 'Future Hire', 'WD_EmployeeStatus'] = dfwd2['Employee Status_y']

col = dfwd2.pop('WD_EmployeeStatus')
dfwd2.insert(2, col.name, col)

dfwd2['Worker Build'] = dfwd2['Worker'].fillna(dfwd2['Worker_x']).fillna(dfwd2['Worker_y'])
dfwd2.insert(3, 'WD_Worker', dfwd2['Worker Build'])
dfwd2 = dfwd2.drop(columns = ['Worker', 'Worker_x', 'Worker_y', 'Worker Build'])

dfwd2['Hire_Date'] = ''

dfwd2.loc[~dfwd2['Hire Date_x'].isnull(), 'Hire_Date'] = dfwd2['Hire Date_x']
dfwd2.loc[~dfwd2['Hire Date'].isnull(), 'Hire_Date'] = dfwd2['Hire Date']
dfwd2.loc[~dfwd2['Hire Date_y'].isnull(), 'Hire_Date'] = dfwd2['Hire Date_y']

dfwd2 = dfwd2.drop(columns = ['Hire Date_x','Hire Date','Hire Date_y','Status Details','Employee Status_x','Employee Status_y'])

col = dfwd2.pop('Hire_Date')
dfwd2.insert(4, 'Hire Date', col)

#dfwd2['Hire Date Build'] = dfwd2['Hire Date_x'].fillna(dfwd2['Hire Date_x']).fillna(dfwd2['Hire Date_y'])
#dfwd2.insert(4, 'Hire_Date', dfwd2['Hire Date Build'])
#dfwd2 = dfwd2.drop(columns = ['Hire Date Build'])

col = dfwd2.pop('Termination Date')
dfwd2.insert(5, col.name, col)

dfwd2['GUID Build'] = dfwd2['GUID'].fillna(dfwd2['GUID_x']).fillna(dfwd2['GUID_y'])
dfwd2.insert(6, 'WD_GUID', dfwd2['GUID Build'])
dfwd2 = dfwd2.drop(columns = ['GUID', 'GUID_x', 'GUID_y', 'GUID Build'])

dfwd2['Line of Service Build'] = dfwd2['Line of Service'].fillna(dfwd2['Cost Center Level 3_x']).fillna(dfwd2['Cost Center Level 3_y']).fillna(dfwd2['PwC Global Line Of Service'])
dfwd2.insert(7, 'LoS', dfwd2['Line of Service Build'])
dfwd2 = dfwd2.drop(columns = ['Line of Service', 'Cost Center Level 3_x', 'Cost Center Level 3_y', 'PwC Global Line Of Service', 'Line of Service Build'])

dfwd2['Job Level Build'] = dfwd2['Job Level - Primary Position_x'].fillna(dfwd2['Job Level - Primary Position_x']).fillna(dfwd2['Job Level'])
dfwd2.insert(7, 'Role Level', dfwd2['Job Level Build'])
dfwd2 = dfwd2.drop(columns = ['Job Level - Primary Position_x', 'Job Level - Primary Position_y', 'Job Level', 'Job Level Build'])

dfwd2['Role LoS Build'] = dfwd2['Role Level'].fillna(dfwd2['LoS'])
dfwd2.insert(7, 'WD_LoS', dfwd2['Role LoS Build'])
dfwd2 = dfwd2.drop(columns = ['Role Level', 'LoS', 'Role LoS Build'])

dfwd2['Cost Center ID Build'] = dfwd2['Cost Center - ID'].fillna(dfwd2['Cost Center ID_x']).fillna(dfwd2['Cost Center ID_y'])
dfwd2.insert(8, 'WD_CC_Code', dfwd2['Cost Center ID Build'])
dfwd2 = dfwd2.drop(columns = ['Cost Center - ID', 'Cost Center ID_x', 'Cost Center ID_y', 'Cost Center ID Build'])

dfwd2['Cost Center Name Build'] = dfwd2['Cost Center Name'].fillna(dfwd2['Cost Center - Name_x']).fillna(dfwd2['Cost Center - Name_y'])
dfwd2.insert(9, 'WD_CC_Name', dfwd2['Cost Center Name Build'])
dfwd2 = dfwd2.drop(columns = ['Cost Center Name', 'Cost Center - Name_x', 'Cost Center - Name_y', 'Cost Center Name Build'])

dfwd2['Company Reference ID Build'] = dfwd2['Company Reference ID_x'].fillna(dfwd2['Company Reference ID_y'])
dfwd2.insert(10, 'WD_CR_Code', dfwd2['Company Reference ID Build'])
dfwd2 = dfwd2.drop(columns = ['Company Reference ID_x', 'Company Reference ID_y', 'Company Reference ID Build'])

dfwd2['Company Hierarchy Build'] = dfwd2['Company Hierarchy_x'].fillna(dfwd2['Company Hierarchy_y'])
dfwd2.insert(11, 'WD_CompanyHierarchy', dfwd2['Company Hierarchy Build'])
dfwd2 = dfwd2.drop(columns = ['Company Hierarchy_x', 'Company Hierarchy_y', 'Company Hierarchy Build'])

dfwd2['Host Country Build'] = dfwd2['Host Country (Alternative)'].fillna(dfwd2['Country'])
dfwd2.insert(12, 'Host Country', dfwd2['Host Country Build'])
dfwd2 = dfwd2.drop(columns = ['Host Country (Alternative)', 'Country', 'Host Country Build'])

dfwd2['Country/Territory Build'] = dfwd2['Country/Territory'].fillna(dfwd2['Host Country / Territory']).fillna(dfwd2['Country/Territory of Work Location'])
dfwd2.insert(12, 'Location', dfwd2['Country/Territory Build'])
dfwd2 = dfwd2.drop(columns = ['Country/Territory', 'Host Country / Territory', 'Country/Territory of Work Location', 'Country/Territory Build'])

if dfwd2['WD_EmployeeStatus'].iteritems() == 'Secondment':
    if dfwd2[(dfwd2['Assignment Reason (Alternative)'] != 'No Longer on Assignment (Long)')]:
        if dfwd2[len(dfwd2['Host Country (Alternative)']) == 0]:
            dfwd2['Territory'] = dfwd2['Host Country (Alternative)']
elif dfwd2['WD_EmployeeStatus'].iteritems() == 'Secondment': 
    if dfwd2[(dfwd2['Assignment Reason (Alternative)'] != 'No Longer on Assignment (Long)')]:
        if dfwd2[len(dfwd2['Host Country / Territory']) == 0]: 
            dfwd2['Territory'] = dfwd2['Host Country / Territory'] 
else:
    dfwd2['Territory'] = dfwd2['WD_CompanyHierarchy'].str.replace('PwC ', '')

dfwd2 = dfwd2.drop(columns = ['Assignment Reason (Alternative)'])
    
dfwd2['Country Build'] = dfwd2['Territory'].fillna(dfwd2['Location']).fillna(dfwd2['Host Country'])
dfwd2.insert(12, 'WD_Country', dfwd2['Country Build'])
dfwd2 = dfwd2.drop(columns = ['Territory', 'Location', 'Host Country', 'Country Build'])

dfwd2['Legal Entity Name Build'] = dfwd2['Company - Name'].fillna(dfwd2['Company - Name_x']).fillna(dfwd2['Company - Name_y'])
dfwd2.insert(13, 'WD_LE_Name', dfwd2['Legal Entity Name Build'])
dfwd2 = dfwd2.drop(columns = ['Company - Name', 'Company - Name_x', 'Company - Name_y', 'Legal Entity Name Build'])

dfwd2['Management Level Build'] = dfwd2['Management Level'].fillna(dfwd2['Management Level_x']).fillna(dfwd2['Management Level_y'])
dfwd2.insert(14, 'WD_Role', dfwd2['Management Level Build'])
dfwd2 = dfwd2.drop(columns = ['Management Level', 'Management Level_x', 'Management Level_y', 'Management Level Build'])

dfwd2['Email Build'] = dfwd2['Work Email'].fillna(dfwd2['Email - Primary Work_x']).fillna(dfwd2['Email - Primary Work_y'])
dfwd2.insert(15, 'WD_Email', dfwd2['Email Build'])
dfwd2 = dfwd2.drop(columns = ['Email Build', 'Work Email', 'Email - Primary Work_x', 'Email - Primary Work_y'])

dfwd2['Worker Type Build'] = dfwd2['Worker Type'].fillna(dfwd2['Worker Type_x']).fillna(dfwd2['Worker Type_y'])
dfwd2.insert(16, 'WD_WorkerType', dfwd2['Worker Type Build'])
dfwd2 = dfwd2.drop(columns = ['Worker Type', 'Worker Type_x', 'Worker Type_y', 'Worker Type Build'])

dfwd2['Job Category Build'] = dfwd2['Job Category'].fillna(dfwd2['Job Category_x']).fillna(dfwd2['Job Category_y'])
dfwd2.insert(17, 'WD_JobCategory', dfwd2['Job Category Build'])
dfwd2 = dfwd2.drop(columns = ['Job Category', 'Job Category_x', 'Job Category_y', 'Job Category Build'])

dfwd2['Contingent Worker Type Build'] = dfwd2['Contingent Worker Type'].fillna(dfwd2['Contingent Worker Type_x']).fillna(dfwd2['Contingent Worker Type_y'])
dfwd2.insert(18, 'WD_CW_Type', dfwd2['Contingent Worker Type Build'])
dfwd2 = dfwd2.drop(columns = ['Contingent Worker Type',
                              'Contingent Worker Type_x',
                              'Contingent Worker Type_y',
                              'Contingent Worker Type Build']).drop_duplicates()


col = dfwd2.pop('Is Inbound IA? (Exception Process)')
dfwd2.insert(19, 'Is IA?', col)

count1 = dfwd2['WD_PartyID'].value_counts()
count1 = pd.DataFrame({'WD_PartyID':count1.index, 'CountPID':count1.values})
dfwd2 = pd.merge(dfwd2, count1, on = 'WD_PartyID', how = 'outer')

count2 = dfwd2['WD_EmployeeID'].value_counts()
count2 = pd.DataFrame({'WD_EmployeeID':count2.index, 'CountEID':count2.values})
dfwd2 = pd.merge(dfwd2, count2, on = 'WD_EmployeeID', how = 'outer')

cols = dfwd2.columns.tolist()

dfwd2 = dfwd2[['WD_PartyID',
 'WD_EmployeeID',
 'WD_EmployeeStatus',
 'WD_Worker',
 'Hire Date',
 'Termination Date',
 'WD_GUID',
 'WD_LoS',
 'WD_CC_Code',
 'WD_CC_Name',
 'WD_CR_Code',
 'WD_CompanyHierarchy',
 'WD_Country',
 'WD_LE_Name',
 'WD_Role',
 'WD_Email',
 'WD_WorkerType',
 'WD_JobCategory',
 'WD_CW_Type',
 'Is IA?',
 'Exists_df1_df2',
 'Exists_dfwd1_df3',
 'CountPID',
 'CountEID']]

#dfwd2['EmpIDCount'] = dfwd2.groupby('')

display(dfwd2)

,WD_EmployeeID,WD_Worker,Hire Date,Termination Date,WD_GUID,WD_LoS,WD_EmployeeStatus,WD_CC_Code,WD_CC_Name,WD_CR_Code,WD_CompanyHierarchy,WD_Country,WD_LE_Name,WD_Role,WD_Email,WD_WorkerType,WD_JobCategory,WD_CW_Type,Is IA?,Exists_df1_df2,Exists_dfwd1_df3,Count
WD_PartyID,,,,,,,,,,,,,,,,,,,,,,
0000fb73-0624-4e47-a87b-72bc7d1da06f,100689987.0,Zezheng Gan,2018-09-01,NaT,zgan005,Assurance,Active,NL12000412,BAS SRG,f30abfab-4c61-4f45-b40c-1ab13706fd3c,PwC Netherlands,Netherlands,PwC Accountants N.V.,Senior Associate,zezheng.gan@pwc.com,Employee,Client Services,NaN,NaN,left_only,left_only,1
000235db-d90f-4881-8774-8bb5c9d8eb76,101074699.0,Simon Letko,2022-09-12,2022-02-28,sletko002,Specialist 5,Active,AT18000019,ASS FS Banking,5715ad3e-bf5d-46a2-a831-e68192667608,PwC Austria,Austria,PwC Österreich GmbH Wirtschaftsprüfungsgesells...,Specialist,simon.l.letko@pwc.com,Employee,Client Services,NaN,NaN,both,left_only,1
0005e477-fa19-406e-b5f5-fea23f60afed,100723663.0,Jan Philipp Boltz [C],2018-07-01,NaT,jboltz002,Internal Firm Services,Active,DE01000238,INFRA FS Berlin,ff6d5af8-9ae6-47d9-8317-eaa8b0e2e7b9,PwC Germany,Germany,PRICEWATERHOUSECOOPERS Gesellschaft mit beschr...,Specialist,jan.philipp.boltz@pwc.com,Contingent Worker,Practice Support,Vendor,NaN,left_only,left_only,1
00066b84-677f-476f-bcda-9e45b2463f30,101231784.0,Marie-Theres Zimmerling,2022-07-01,NaT,mzimmerlin002,Advisory,Active,DE01000792,ADV BC Risk & Regulatory,ff6d5af8-9ae6-47d9-8317-eaa8b0e2e7b9,PwC Germany,Germany,PRICEWATERHOUSECOOPERS Gesellschaft mit beschr...,Associate,marie-theres.zimmerling@pwc.com,Employee,Client Services,NaN,NaN,left_only,left_only,1
00071b71-c5d2-4f77-b995-db14fb54675f,101110909.0,Lia Ballach,2021-10-01,NaT,lballach001,Internal Firm Services,Active,DE01000015,HC Human Capital,ff6d5af8-9ae6-47d9-8317-eaa8b0e2e7b9,PwC Germany,Germany,PRICEWATERHOUSECOOPERS Gesellschaft mit beschr...,Senior Associate,lia.ballach@pwc.com,Employee,Practice Support,NaN,NaN,left_only,left_only,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff6ad920-d740-45f6-9d0e-93f23b8810e5,101279003.0,Eva Maria Sacco,2023-01-01,NaT,NaN,Internal Firm Services,Future Hire,DE01000121,FIN Treasury Germany,ff6d5af8-9ae6-47d9-8317-eaa8b0e2e7b9,PwC Germany,Germany,PRICEWATERHOUSECOOPERS Gesellschaft mit beschr...,Senior Manager,NaN,Employee,Practice Support,NaN,No,NaN,right_only,1
ff7616e0-e202-449e-a38b-f4d3b4c7c1a0,101304636.0,Leo Horlitz,2022-11-01,NaT,lhorlitz001,ADV Strategy,Future Hire,DEZ1010001,ADV SC DE Consulting,5cc1b3b6-8a15-47d6-b5ad-600d68e496a9,PwC Germany,Germany,PwC Strategy& Germany GmbH,Associate,leo.horlitz@pwc.com,Employee,Client Services,NaN,No,NaN,right_only,1
ff780117-8338-4dc3-b1f6-b4d462a41471,101247147.0,Julius Turschner,2022-10-01,NaT,jturschner001,Assurance,Future Hire,DE01000243,ASR IS Ost,ff6d5af8-9ae6-47d9-8317-eaa8b0e2e7b9,PwC Germany,Germany,PRICEWATERHOUSECOOPERS Gesellschaft mit beschr...,Intern/Trainee,NaN,Employee,Client Services,NaN,No,NaN,right_only,1


In [4]:
dfwd2.to_excel('DataMergeSAPOuterMergedFields.xlsx')

In [7]:
print('\n'.join([''.join([('PwC'[(x-y)%3] if((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3<=0 else' ') for x in range(-30,30)]) for y in range(15,-15,-1)]))

                                                            
                                                            
                                                            
                wCPwCPwCP           PwCPwCPwC               
            wCPwCPwCPwCPwCPwC   PwCPwCPwCPwCPwCPw           
          PwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPw         
         PwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCP        
        PwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwC       
        wCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCP       
        CPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPw       
        PwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwC       
        wCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCP       
        CPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPw       
         wCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPw        
          PwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPw         
          wCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwCPwC         
            wCPwCPwCPwCP